In [ ]:
# Get offers from database
from offer import Offer

Offer.ConnectToDatabase()
Offer.SetKeyspace("symplicity")
Offer.PrepareStatements()

career = "COMUNICACIÓN AUDIOVISUAL"
min_date = (1, 2013)
max_date = (12, 2017)
source = "symplicity"
text_fields = ["Description", "Job Title", "Qualifications"]

offers = [offer for offer in Offer.ByDateRange(min_date, max_date, source) if career in offer.careers]

import csv
with open('data.csv', 'w') as csvfile:
    fieldnames = text_fields
    wr = csv.DictWriter(csvfile, fieldnames=fieldnames)
    wr.writeheader()
    
    for offer in offers:        
        row = {}
        for field in fieldnames:
            row[field] = offer.features[field]
        wr.writerow(row)

In [2]:
import csv
data = []
with open('data.csv') as csvfile:
    reader = csv.DictReader(csvfile)    
    for row in reader:
        text = " ".join(row.values())
        data.append(text)


In [6]:
import nltk
from nltk.tag.stanford import StanfordPOSTagger

spanish_postagger = StanfordPOSTagger('models/spanish.tagger', './stanford-postagger.jar')

/home/howl24/.virtualenvs/TextClassification/lib/python3.5/site-packages/nltk/tag/stanford.py:149: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordPOSTagger, self).__init__(*args, **kwargs)


In [7]:
punctuations = ['•','/']

translator = str.maketrans("".join(punctuations),' '*len(punctuations))
proc_data = []
for text in data:
    text = text.lower()
    text = text.translate(translator)
    proc_data.append(text)
    
data = proc_data


In [8]:
data[0]

'trainee de calidad y rse  \tdesarrollar los informes mensuales que se entrega a los clientes.\n \tdesarrollar la comunicación que llega a los clientes.\n \tapoyar en la optimización de manuales, procesos y procedimientos a fin de cumplir con las disposiciones de la empresa.\n \tvelar por el adecuado uso de la imagen corporativa.\n \tdesarrollar con las coordinaciones necesarias el contenido de la documentación relativa a comunicación general, tutoriales y manuales de diversas áreas.\n \tatender y orientar correcta y oportunamente al cliente interno en cuanto a uso de elementos propios de la imagen y redacción de textos.  \n \tmantener y proponer mejoras que impacten en la calidad de la comunicación del servicio ofrecido al cliente interno y externo.\n \tsugerir la participación de la empresa en actividades culturales, cívicas, artísticas y de proyección social, de acuerdo a los lineamientos aprobados, que incentiven el desarrollo de la comunidad, proyectando la imagen y presencia de l

In [9]:
from nltk.tokenize import word_tokenize
from nltk.tag.stanford import StanfordPOSTagger
import multiprocessing

def divide_list(n_chunks, list_to_divide):
    chunk_size = len(list_to_divide)//n_chunks + 1
    chunks = [list_to_divide[i:i+chunk_size] for i in range(0, len(list_to_divide), chunk_size)]

    return chunks

def get_pos_tag(data, q_terms):    
    spanish_postagger = StanfordPOSTagger('models/spanish.tagger', './stanford-postagger.jar')
    
    terms_by_tag = {}
    terms_by_tag['n'] = set()
    terms_by_tag['v'] = set()
    terms_by_tag['a'] = set()
    terms_by_tag['c'] = set()
    terms_by_tag['d'] = set()
    terms_by_tag['f'] = set()
    terms_by_tag['i'] = set()
    terms_by_tag['p'] = set()
    terms_by_tag['r'] = set()
    terms_by_tag['s'] = set()
    terms_by_tag['w'] = set()
    terms_by_tag['z'] = set()

    for text in data:
        tag_terms = spanish_postagger.tag(word_tokenize(text))
        for term in tag_terms:            
            for tag in terms_by_tag:
                if term[1][0] == tag:
                    terms_by_tag[tag].add(term[0])

    print("El proceso ha terminado")
    q_terms.put(terms_by_tag)


q_terms = multiprocessing.Queue()
n_jobs = 4
jobs = []
chunks = divide_list(n_jobs, data)

for chunk in chunks:
    p = multiprocessing.Process(target = get_pos_tag,
                                args = (chunk, q_terms))
    
    jobs.append(p)
    p.start()
    
for proc in jobs:
    proc.join()
    
    
terms_by_tag = q_terms.get()
for i in range(n_jobs-1):
    new_terms = q_terms.get()
    for tag in terms_by_tag:
        terms_by_tag[tag].update(new_terms[tag])
        

/home/howl24/.virtualenvs/TextClassification/lib/python3.5/site-packages/nltk/tag/stanford.py:149: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordPOSTagger, self).__init__(*args, **kwargs)
/home/howl24/.virtualenvs/TextClassification/lib/python3.5/site-packages/nltk/tag/stanford.py:149: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordPOSTagger, self).__init__(*args, **kwargs)
/home/howl24/.virtualenvs/TextClassification/lib/python3.5/site-packages/nltk/tag/stanford.py:149: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordPOSTagger, self).__init__(*args, **kwargs)
/home/howl24/

KeyboardInterrupt: 

  File "/home/howl24/.virtualenvs/TextClassification/lib/python3.5/site-packages/nltk/tag/stanford.py", line 89, in tag_sents
    config_java(options=self.java_options, verbose=False)
  File "/home/howl24/.virtualenvs/TextClassification/lib/python3.5/site-packages/nltk/internals.py", line 131, in java
    (stdout, stderr) = p.communicate()
  File "/home/howl24/.virtualenvs/TextClassification/lib/python3.5/site-packages/nltk/tag/stanford.py", line 84, in tag
    return sum(self.tag_sents([tokens]), [])
  File "/home/howl24/.virtualenvs/TextClassification/lib/python3.5/site-packages/nltk/tag/stanford.py", line 107, in tag_sents
    stdout=PIPE, stderr=PIPE)
  File "/usr/lib/python3.5/subprocess.py", line 1072, in communicate
    stdout, stderr = self._communicate(input, endtime, timeout)
  File "/home/howl24/.virtualenvs/TextClassification/lib/python3.5/site-packages/nltk/internals.py", line 131, in java
    (stdout, stderr) = p.communicate()
  File "/home/howl24/.virtualenvs/TextClassif

In [26]:
len(terms_by_tag)

12

In [27]:
for tag in terms_by_tag:
    if "procesamiento" in terms_by_tag[tag]:
        print(tag)

n


In [28]:
vocab = {}
vocab.update(dict.fromkeys(terms_by_tag['n'], 0))
vocab.update(dict.fromkeys(terms_by_tag['v'], 0))
vocab.update(dict.fromkeys(terms_by_tag['a'], 0))
len(vocab)

747

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words = ngram_range=(1, 3), min_df=0.0007)
vectorizer.fit(data)
vect_vocab = vectorizer.get_feature_names()
#vect_vocab = set(vect_vocab)
len(vect_vocab)


45293

In [ ]:
# Multiprocessing vocabulary search

def filter_vocabs(vectorizer_vocabs, q_vocabs):
    nsn_filter()
    
q_vocabs = multiprocessing.Queue()
n_jobs = 4
filter_jobs = []
vocab_chunks = divide_list(n_jobs, vect_vocab)

for chunk in vocab_chunks:
    p = multiprocessing.Process(target = filter_vocabs,
                                args = (chunk, q_vocabs))
    
    filter_jobs.append(p)
    p.start()
    
for proc in filter_jobs:
    proc.join()       
       

In [33]:
# n1 + s + n2
# base de datos
for n1 in terms_by_tag['n']:
    for s in terms_by_tag['s']:
        for n2 in terms_by_tag['n']:            
            phrase = " ".join([n1, s, n2])
            if phrase in vect_vocab:                
                vocab.add(" ".join([n1, s, n2]))

proceso de integración
proceso de modernización
consultorías de m
valorización de empresas
toma de decisiones
historial de éxito
gestión por procesos
gestión para toma
gestión de procesos
gestión ( erp
temas de gestión
temas de impuestos
reportes de gestión
reportes de comisiones
reportes de eventos
comisiones de fin
fin de mes
revisión de indicadores
revisión de estrategias
revisión de modelos
revisión de documentación
satisfacción del cliente
exterior de clientes
roles de agencia
cumplimiento del sílabo
cumplimiento de acuerdos
carreras de administración
presentación al directorio
proyectos de delivery
proyectos de implementación
proyectos de lanzamiento
proyectos de infraestructura
coordinador de gestión
coordinador de monitoreo
prácticas de gobierno
organización de rrhh
nivel de servicio
trayectoria en cadena
productividad de belcorp
información de negociación
información de compras
ingles a nivel
head de operaciones
empresas del perú
director de monitoreo
estructura de capital
act

In [18]:
# n + c + n
# base de datos
for n1 in terms_by_tag['n']:
    for c in terms_by_tag['c']:
        for n2 in terms_by_tag['n']:           
            phrase = " ".join([n1, c, n2])
            if phrase in vect_vocab:                
                vocab.add(" ".join([n1, c, n2]))

gestión y experiencia
operaciones y producto
controles y gestión
procesamiento y análisis
stakeholders y riesgos
lectura y escritura
vida y presencia
administración y dirección
gobierno y adherencia
empresas y unidades
monitoreo y publicidad
acuerdos y objetivos
departamento y equipo
negociación y precio
innovación y costo
financiamiento y estructura
diarios y revistas
administracion y finanzas
crecimiento y productividad
industrias y funciones
procedimientos y controles
sistema y registro
sistemas y conocimiento
importaciones y conocimiento
volumen y desarrollo
resultados según contrato
registro y contabilidad
valores y fideicomisos
procesos y calidad
procesos y medidas
diseño o rediseño
difusión y educación
experiencia como analista
ventas y distribución
gerencia o jefatura
seguimiento e implementación
bienes y servicios
traspasos y afiliaciones
propuestas y seguimiento
auditorías y exámenes
modificación y aprobación
tecnología y outsourcing
directorios o gerencias
preparación y actu

In [25]:
# n + c + n + a
# base de datos
for n1 in terms_by_tag['n']:
    for c in terms_by_tag['c']:
        for n2 in terms_by_tag['n']:
            for a in terms_by_tag['a']:
                phrase = " ".join([n1, c, n2, a])
                if phrase in vect_vocab:
                    print(phrase)
                    vocab.add(phrase)

innovación y costo total
administracion y finanzas mínimo
importaciones y conocimiento básicos
bienes y servicios requeridos
traspasos y afiliaciones profuturo
propuestas y seguimiento logístico
auditorías y exámenes especiales
spss o paquetes similares
oportunidades y fortalezas observadas
finanzas y gobierno corporativo
compras y contratos mínimo
macros y tablas dinámicas
normas y disposiciones vigentes


In [26]:
# v + s + n
# base de datos
for v in terms_by_tag['v']:
    for s in terms_by_tag['s']:
        for n in terms_by_tag['n']:
            phrase = " ".join([v, s, n])
            if phrase in vect_vocab:
                print(phrase)
                vocab.add(phrase)

eps en base
entrega de materiales
dictado de programas
traduzcan en acciones
encargue de conformidad
core del negocio
avanzado en conversación
avanzado de excel
avanzado de programas
participar en consultorías
acompañar al usuario
office ( manejo
aborda a través
detallado en tiempos
utilizados de forma
convertirse en empresas
partir de fuentes
trabajar para accenture
procesada en conjunto


In [29]:
# v + n
# Recopilar información
for v in terms_by_tag['v']:    
    for n in terms_by_tag['n']:
        phrase = " ".join([v, n])
        if phrase in vect_vocab:
            print(phrase)
            vocab.add(phrase)

excel )
establecer soluciones
identificar ámbitos
analizar condiciones
analizar indicadores
ver archivo
formular informes
finance )
atender necesidades
desarrollar modelos
ejecutar proyectos
ejecutar acciones
manejan fideicomisos
core business
generen sinergias
proponer actividades
proponiendo recomendaciones
administrative agent
avanzado (
conocer uso
realizar análisis
brindar asesoría
dar seguimiento
realiza seguimiento
generar conclusiones
generar análisis
generar productos
office (
realizan estudios
realizado proyectos
dando cuenta
absolver consultas
generando planes
sbs (
mitiguen riesgos
brindan servicio
intelligence office


In [30]:
# n + a
# Recopilar información
for n in terms_by_tag['n']:
    for a in terms_by_tag['a']:
        phrase = " ".join([n, a])
        if phrase in vect_vocab:
            print(phrase)
            vocab.add(phrase)

proceso precontractual
fuentes digitales
gestión pública
gestión técnica
gestión presupuestaria
operaciones diarias
reportes financieros
reportes coyunturales
controles internos
hechos irregulares
venta asignados
plan curricular
proyectos especiales
proyectos diversos
coordinador académico
empresa global
empresa transnacional
gobierno corporativo
gobierno regional
nivel intermedio
nivel nacional
condiciones óptimas
condiciones estipuladas
información pública
información cualitativa
información social
contratos realizados
contratos correspondientes
contratos mínimo
empresas consultoras
empresas industriales
actividades necesarias
actividades operativas
ingeniería industrial
decisiones gerenciales
mercados financieros
región andina
abastecimiento sostenible
simplificación administrativa
gasto mensual
gerente comercial
ejercicio consolidado
conocimientos especificos
conocimientos avanzados
conocimientos indispensables
informe anual
conocimiento básicos
conocimiento básico
exámenes especia

In [47]:
# n + a + c + a
# Recopilar información
for n in terms_by_tag['n']:
    for a1 in terms_by_tag['a']:
        for c in terms_by_tag['c']:
            for a2 in terms_by_tag['a']:
                phrase = " ".join([n, a1, c, a2])
                if phrase in vect_vocab:
                    print(phrase)
                    vocab.add(phrase)

información cualitativa y cuantitativa
estudios sociales y económicos
entidades públicas y privadas
( locales y globales
datos cualitativos y cuantitativos


In [22]:
for w in vocab:
    if len(w.split()) > 1:
        print(w)
    

manejo de caja
manejo de sap
gestión de procesos
temas de gestión
) a nivel
análisis de rentabilidad
funcionario de negocios
asistente de operaciones
mercado de capitales
programas de incentivos
área de créditos
productos del proyecto
canales de venta
entrega de información
equipo de ventas
flujo de caja
toma de decisiones
trabajo en equipo
as a team
acciones de marketing
desarrollo de proyectos
años como administrador
áreas de negocio
gerente de ventas
manejo de base
requerimientos de información
office ( word
procesos de control
analista de compras
cobranzas en empresas
interpretación de estados
equipo de cobranzas
especialización en gestión
años como analista
horario de trabajo
carrera de administración
soporte al ejecutivo
propuestas de mejora
análisis de información
oportunidades de negocio
curso de especialización
análisis de indicadores
clientes de acuerdo
años como mínimo
áreas de créditos
bases de datos
capacidad de análisis
nivel de comunicación
dominio de inglés
elaboración 

In [13]:
for term in vocab:
    if term not in vect_vocab:
        print(term)

In [22]:
for term in vect_vocab:
    if term not in vocab:
        print(term)

&
,
-
.
02
03
1.asistir
10
12
150
2.
281.000
3
3.
4
90
:
;
a
al
bajo
con
contra
d
de
del
desde
dos
durante
e
en
entre
etc
hasta
kardex
o
otass
para
por
según
sobre
y
–
